In [1]:
from __future__ import print_function
from __future__ import unicode_literals
from __future__ import division

In [2]:
import numpy as np
import pandas as pd

In [4]:
import datetime as dt

In [3]:
from pyspark.sql import Row

In [5]:
spark

# Definition

In [39]:
start_ts = dt.datetime.strptime("2017-01-01 00:00:00", '%Y-%m-%d %H:%M:%S')
end_ts = dt.datetime.strptime("2017-08-20 23:59:59", '%Y-%m-%d %H:%M:%S')

In [43]:
(end_ts - start_ts).days // 7

33

In [50]:
(end_ts.weekday() + 1) % 7

0

# Load data

In [6]:
header = sc.textFile('kktv-17-11/train').take(1)[0]

In [7]:
header

u'user_id,device_id,session_id,title_id,event_time,played_duration,action_trigger,platform,episode_number,series_total_episodes_count,internet_connection_type,is_trailer'

In [8]:
train_evts = sc.textFile('kktv-17-11/train').filter(lambda e: e != header)

In [9]:
train_evts.count()

57661193

In [10]:
header.split(',')

[u'user_id',
 u'device_id',
 u'session_id',
 u'title_id',
 u'event_time',
 u'played_duration',
 u'action_trigger',
 u'platform',
 u'episode_number',
 u'series_total_episodes_count',
 u'internet_connection_type',
 u'is_trailer']

In [51]:
def parse_slot(d):
    slot = 3
    week_day = d.weekday()
    
    if d < dt.datetime(d.year, d.month, d.day, 1, 0, 0):
        slot = 3
    elif d < dt.datetime(d.year, d.month, d.day, 9, 0, 0):
        slot = 0
    elif d < dt.datetime(d.year, d.month, d.day, 17, 0, 0):
        slot = 1
    elif d < dt.datetime(d.year, d.month, d.day, 21, 0, 0):
        slot = 2
    
    if slot == 3:
        week_day -= 1
    
    return slot + d.weekday() * 4

In [54]:
def parse_evt(p):
    evt_time = dt.datetime.strptime(p[4][:19], '%Y-%m-%d %H:%M:%S')
    time_diff = evt_time - start_ts 

    r = Row(
        user_id=p[0],
        device_id=p[1],
        session_id=p[2],
        title_id=p[3],
        event_time=evt_time,
        played_duration=float(p[5]),
        action_trigger=p[6],
        platform=p[7],
        episode_number=int(p[8]),
        series_total_episodes_count=int(p[9]),
        internet_connection_type=p[10],
        is_trailer=bool(p[11]),
        month=evt_time.month,
        day=evt_time.day,
        hour=evt_time.hour,
        week = (evt_time - start_ts).days // 7,
        slot=parse_slot(evt_time)
    )
    return r

In [55]:
train_df = train_evts.map(lambda e: e.split(',')).map(parse_evt).toDF()

In [56]:
train_df.printSchema()

root
 |-- action_trigger: string (nullable = true)
 |-- day: long (nullable = true)
 |-- device_id: string (nullable = true)
 |-- episode_number: long (nullable = true)
 |-- event_time: timestamp (nullable = true)
 |-- hour: long (nullable = true)
 |-- internet_connection_type: string (nullable = true)
 |-- is_trailer: boolean (nullable = true)
 |-- month: long (nullable = true)
 |-- platform: string (nullable = true)
 |-- played_duration: double (nullable = true)
 |-- series_total_episodes_count: long (nullable = true)
 |-- session_id: string (nullable = true)
 |-- slot: long (nullable = true)
 |-- title_id: string (nullable = true)
 |-- user_id: string (nullable = true)
 |-- week: long (nullable = true)



In [59]:
train_df.select('user_id', 'event_time', 'month', 'day', 'hour', 'week', 'slot').show(200)

+-------+-------------------+-----+---+----+----+----+
|user_id|         event_time|month|day|hour|week|slot|
+-------+-------------------+-----+---+----+----+----+
|      0|2017-06-08 14:54:30|    6|  8|  14|  22|  13|
|      0|2017-06-08 14:54:34|    6|  8|  14|  22|  13|
|      0|2017-06-08 14:54:36|    6|  8|  14|  22|  13|
|      0|2017-06-08 14:54:38|    6|  8|  14|  22|  13|
|      0|2017-06-08 14:54:40|    6|  8|  14|  22|  13|
|      0|2017-06-08 14:54:41|    6|  8|  14|  22|  13|
|      0|2017-06-08 14:54:46|    6|  8|  14|  22|  13|
|      0|2017-06-08 14:57:26|    6|  8|  14|  22|  13|
|      0|2017-06-08 15:18:25|    6|  8|  15|  22|  13|
|      0|2017-06-08 15:19:52|    6|  8|  15|  22|  13|
|      0|2017-06-08 15:19:56|    6|  8|  15|  22|  13|
|      0|2017-06-08 15:19:59|    6|  8|  15|  22|  13|
|      0|2017-06-08 15:20:03|    6|  8|  15|  22|  13|
|      0|2017-06-08 15:20:05|    6|  8|  15|  22|  13|
|      0|2017-06-08 15:20:20|    6|  8|  15|  22|  13|
|      0|2

In [35]:
train_df.selectExpr('max(event_time)', 'min(event_time)').show()

+-------------------+-------------------+
|    max(event_time)|    min(event_time)|
+-------------------+-------------------+
|2017-08-20 23:59:59|2017-01-01 00:00:00|
+-------------------+-------------------+



# Set Time Range:  2017/1/2 - 2017/8/13

In [65]:
train_slot_df = train_df.groupby('user_id', 'week', 'slot').count()

In [66]:
train_slot_df.show()

+-------+----+----+-----+
|user_id|week|slot|count|
+-------+----+----+-----+
|     10|  21|  14|   19|
|     11|  13|  20|    2|
|     13|  22|  22|   30|
|     13|  32|   8|   15|
|     19|  31|   2|   17|
|     20|  20|  17|   13|
|     21|  25|   6|    2|
|     26|  14|   0|    5|
|     26|  24|   8|   36|
|     34|  21|   9|   15|
|     38|  32|  24|    2|
|     39|  28|   8|    4|
|     43|  21|   4|   48|
|     45|  21|   8|    7|
|     54|  31|   1|    2|
|     61|  20|   1|    2|
|     83|  11|  26|  138|
|     83|  23|   5|   20|
|     91|   6|   3|    1|
|     99|  30|  13|    2|
+-------+----+----+-----+
only showing top 20 rows



In [67]:
train_slot_df.write.format('parquet').save('tmp/train_slot')